# Homework 1: Statistics with Python and SQL

### Setup

In [2]:
%load_ext sql

In [4]:
%sql sqlite:///sqlite-dvdrental.db

In [20]:
import sqlalchemy
import pandas as pd
import seaborn as sns

In [21]:
con = sqlalchemy.create_engine("sqlite:///sqlite-dvdrental.db")

### 1. Write a SQL query that gives for each customer the average rental duration and the number of rentals.

In [32]:
df = pd.read_sql("""
SELECT
    customer_id
	, c.first_name || ' '|| c.last_name as full_name
	, CAST(AVG(JULIANDAY(r.return_date) - JULIANDAY(r.rental_date))*60 AS int) as duration_in_hours
    , COUNT(rental_id) AS n_rentals
FROM rental r
LEFT JOIN customer c USING (customer_id)
GROUP BY 1, 2
ORDER BY 4 DESC
""", con)

In [33]:
df.head(5)

,customer_id,full_name,duration_in_hours,n_rentals
0,148,ELEANOR HUNT,317,46
1,526,KARL SEAL,352,45
2,144,CLARA SHAW,335,42
3,236,MARCIA DEAN,301,42
4,75,TAMMY SANDERS,299,41
